<a href="https://colab.research.google.com/github/githublior/Monet_cyclegan/blob/main/final_v_Unet_pretrain_1_cyclegan_p2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

path = "/content/drive/MyDrive/Colab Notebooks/deep learning Blutman (CGAN)/data"

Mounted at /content/drive


In [ ]:
import torch
import random, torch, os, numpy as np
import torch.nn as nn
import torch.nn.functional as F
import copy
from PIL import Image
import os
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim as optim
import numpy as np
import torchvision.models
from torchvision.utils import save_image
import torchvision.transforms
import random
import time
import sys

from tqdm import tqdm

from keras.preprocessing.image import ImageDataGenerator
from skimage import io




In [ ]:
#config

# import torch
import albumentations as A
# from albumentations.pytorch import ToTensorV2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
pre_dir  = path + "/301_pic"
pre_m_dir = path + "/aug_monet"
TRAIN_DIR = path +"/data/train"
VAL_DIR = path + "/data/dev"
BATCH_SIZE = 3
LEARNING_RATE_G = 2e-4
LEARNING_RATE_D = 2e-4
LAMBDA_IDENTITY = 5
LAMBDA_CYCLE = 0.5
NUM_WORKERS = 1
WD =  1e-4
NUM_EPOCHS = 10
LOAD_MODEL = False
SAVE_MODEL = True
PRE_G = True
CHECKPOINT_GEN_U = path+"/genu.pth.tar"

CHECKPOINT_GEN_H = path+"/genh.pth.tar"
CHECKPOINT_GEN_Z = path+"/genz.pth.tar"

CHECKPOINT_CRITIC_H = path+"/critich.pth.tar"
CHECKPOINT_CRITIC_Z = path+"/criticz.pth.tar"


# data augmentation
transforms = A.Compose(
    [
        # A.Resize(width=256, height=256),
        # A.HorizontalFlip(p=0.5),
        # A.RandomSizedCrop([180, 230], 256, 256, w2h_ratio=1.0, p = 0.7),
        A.Normalize(mean=[0, 0, 0], std=[0.5, 0.5, 0.5], max_pixel_value=255),
        # ToTensor(),
    ],
    additional_targets={"image0": "image"},
)


In [ ]:
# #utils
# import random, torch, os, numpy as np
# import torch.nn as nn
# # import config
# import copy

def Average(lst):
    return sum(lst) / len(lst)

def save_checkpoint(model, optimizer, filename= path+"/my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
    }
    torch.save(checkpoint, filename)



def load_checkpoint(checkpoint_file, model, optimizer, lr):
    print("=> Loading checkpoint")
    checkpoint = torch.load(checkpoint_file, map_location=DEVICE)
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    # If we don't do this then it will just have learning rate of old checkpoint
    # and it will lead to many hours of debugging \:
    for param_group in optimizer.param_groups:
        param_group["lr"] = lr


def seed_everything(seed=12):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def gen_monet():
  datagen = ImageDataGenerator(
          featurewise_center = True,
          featurewise_std_normalization = True,
          shear_range = 0.2,
          horizontal_flip = True,
          vertical_flip = True,
          brightness_range = (0.5, 1.5))

  image_directory =  path + '/30_monet' #r'C:Users---train/'
  SIZE = 256
  dataset = []
  my_images = os.listdir(image_directory)
  for i, image_name in enumerate(my_images):
      if (image_name.split('.')[1] == 'jpg'):
          image = io.imread(image_directory + '/'+image_name)
          image = Image.fromarray(image, 'RGB')
          image = image.resize((SIZE,SIZE))
          dataset.append(np.array(image))
  x = np.array(dataset)
  i = 0
  for batch in datagen.flow(x, batch_size=1,
                            save_to_dir= path + '/aug_monet' ,save_prefix='dr',save_format='jpg'):
      i += 1
      if i > 300:
        print('here')
        break
  # r'C:Users---Augmented-images',

def select_some_pic():

  image_directory =  path + '/photo_jpg' #r'C:Users---train/'
  output_dir1 = path + "/300_pic"
  output_dir2 = path + "/301_pic"

  # random.shuffle(image_directory)
  my_images = os.listdir(image_directory)
  for i, filename in enumerate(my_images):
    img = Image.open(os.path.join(image_directory, filename)) # images are color images
    # print(i)
    if i <300 : img.save(output_dir1+'/'+filename)
    elif i <600: img.save(output_dir2+'/'+filename)
    else: break
      # if (image_name.split('.')[1] == 'jpg'):
      # print(i)

## takes in a module and applies the specified weight initialization
def weights_init_normal(m):
    '''Takes in a module and initializes all linear layers with weight
        values taken from a normal distribution.'''

    classname = m.__class__.__name__
    # for every Linear layer in a model
    if classname.find('Linear') != -1:
        y = m.in_features
    # m.weight.data shoud be taken from a normal distribution
        m.weight.data.normal_(0.0,0.02)#/np.sqrt(y))
    # m.bias.data should be 0
        m.bias.data.fill_(0)

In [ ]:
# #dataset
# from PIL import Image
# import os
# from torch.utils.data import Dataset
# import numpy as np
from IPython.display import Image as Im

class PictureMonetDataset(Dataset):
    def __init__(self, root_monet, root_picture, transform=None):
        self.root_monet = root_monet
        self.root_picture = root_picture
        self.transform = transform

        self.monet_images = os.listdir(root_monet)
        self.picture_images = os.listdir(root_picture)
        self.length_dataset = max(len(self.monet_images), len(self.picture_images))
        self.monet_len = len(self.monet_images)
        self.picture_len = len(self.picture_images)

    def __len__(self):
        return self.length_dataset
# original
    def __getitem__(self, index):
        monet_img = self.monet_images[index % self.monet_len]
        picture_img = self.picture_images[index % self.picture_len]

        monet_path = os.path.join(self.root_monet, monet_img)
        picture_path = os.path.join(self.root_picture, picture_img)


        monet_img = np.array(Image.open(monet_path))#.convert("RGB"))
        picture_img = np.array(Image.open(picture_path))#.convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=monet_img, image0=picture_img)
            monet_img = augmentations["image"]
            picture_img = augmentations["image0"]

        # print('zb', monet_path, monet_img.shape )
        monet_img =  np.moveaxis(monet_img, -1,0).astype(float)  # change dimension shape : from  [256,256,3] to [3,256,256] to
        # print('bi',monet_path, monet_img.shape , type(monet_img))
        # np.save("/content", monet_img)
        # Im(monet_img)
        picture_img = np.moveaxis(picture_img, -1,0).astype(float)



        return monet_img, picture_img



In [ ]:
# # gerator_model

# import torch
# import torch.nn as nn
# import torch.nn.functional as F

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, use_act=True, **kwargs):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, padding_mode="reflect", **kwargs)
            if down
            else nn.ConvTranspose2d(in_channels, out_channels, **kwargs),
            nn.InstanceNorm2d(out_channels),
            nn.ReLU(inplace=True) if use_act else nn.Identity()
        )

    def forward(self, x):
        return self.conv(x)

class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            ConvBlock(channels, channels, kernel_size=3, padding=1),
            ConvBlock(channels, channels, use_act=False, kernel_size=3, padding=1),
        )

    def forward(self, x):
        return x + self.block(x)

class Generator(nn.Module):
    def __init__(self, img_channels, num_features = 16, num_residuals=9):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(img_channels, num_features, kernel_size=7, stride=1, padding=3 ,padding_mode="reflect"),
            nn.InstanceNorm2d(num_features),
            nn.ReLU(inplace=True),
        )
        self.down_blocks = nn.ModuleList(
            [
                ConvBlock(num_features, num_features*2, kernel_size=3, stride=2, padding=1),
                ConvBlock(num_features*2, num_features*4, kernel_size=3, stride=2, padding=1),
            ]
        )
        self.res_blocks = nn.Sequential(
            *[ResidualBlock(num_features*4) for _ in range(num_residuals)]
        )
        self.up_blocks = nn.ModuleList(
            [
                ConvBlock(num_features*4, num_features*2, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
                ConvBlock(num_features*2, num_features*1, down=False, kernel_size=3, stride=2, padding=1, output_padding=1),
            ]
        )

        self.last = nn.Conv2d(num_features*1, img_channels, kernel_size=7, stride=1, padding=3, padding_mode="reflect")

    def forward(self, x):
        x = self.initial(x)
        for layer in self.down_blocks:
            x = layer(x)
        x = self.res_blocks(x)
        for layer in self.up_blocks:
            x = layer(x)
        return torch.tanh(self.last(x))

def test():
    img_channels = 3
    img_size = 256
    x = torch.randn((2, img_channels, img_size, img_size))
    gen = Generator(img_channels, 9)
    print(gen(x).shape)

if __name__ == "__main__":
    test()



# norm_layer = nn.InstanceNorm2d
# class ResBlock(nn.Module):
#     def __init__(self, f):
#         super(ResBlock, self).__init__()
#         self.conv = nn.Sequential(nn.Conv2d(f, f, 3, 1, 1), norm_layer(f), nn.ReLU(),
#                                   nn.Conv2d(f, f, 3, 1, 1))
#         self.norm = norm_layer(f)
#     def forward(self, x):
#         return F.relu(self.norm(self.conv(x)+x))

# class Generator(nn.Module):
#     def __init__(self, f=64, blocks=6):
#         super(Generator, self).__init__()
#         layers = [nn.ReflectionPad2d(3),
#                   nn.Conv2d(  3,   f, 7, 1, 0), norm_layer(  f), nn.ReLU(True),
#                   nn.Conv2d(  f, 2*f, 3, 2, 1), norm_layer(2*f), nn.ReLU(True),
#                   nn.Conv2d(2*f, 4*f, 3, 2, 1), norm_layer(4*f), nn.ReLU(True)]
#         for i in range(int(blocks)):
#             layers.append(ResBlock(4*f))
#         layers.extend([
#                 nn.ConvTranspose2d(4*f, 4*2*f, 3, 1, 1), nn.PixelShuffle(2), norm_layer(2*f), nn.ReLU(True),
#                 nn.ConvTranspose2d(2*f,   4*f, 3, 1, 1), nn.PixelShuffle(2), norm_layer(  f), nn.ReLU(True),
#                 nn.ReflectionPad2d(3), nn.Conv2d(f, 3, 7, 1, 0),
#                 nn.Tanh()])
#         self.conv = nn.Sequential(*layers)

#     def forward(self, x):
#         return self.conv(x)


# def test():
#     img_channels = 3
#     img_size = 256
#     x = torch.randn((2, img_channels, img_size, img_size))
#     # t = torchvision.transforms.ToTensor()
#     # x = np.array(Image.open(TRAIN_DIR+"/monets/1.jpg").convert("RGB"))
#     # x =t(x)
#     gen = Generator(img_channels, 9)
#     print(gen(x).shape)

# if __name__ == "__main__":
#     test()


torch.Size([2, 3, 256, 256])


In [ ]:
# Generator unet from colab
#https://colab.research.google.com/github/usuyama/pytorch-unet/blob/master/pytorch_unet_resnet18_colab.ipynb#scrollTo=b8EJl0hcC5DH

def convrelu(in_channels, out_channels, kernel, padding):
  return nn.Sequential(
    nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
    nn.ReLU(inplace=True),
  )


class ResNetUNet(nn.Module):
  def __init__(self, n_class):
    super().__init__()

    self.base_model = torchvision.models.resnet18(pretrained=False)
    self.base_layers = list(self.base_model.children())

    self.layer0 = nn.Sequential(*self.base_layers[:3]) # size=(N, 64, x.H/2, x.W/2)
    self.layer0_1x1 = convrelu(64, 64, 1, 0)
    self.layer1 = nn.Sequential(*self.base_layers[3:5]) # size=(N, 64, x.H/4, x.W/4)
    self.layer1_1x1 = convrelu(64, 64, 1, 0)
    self.layer2 = self.base_layers[5]  # size=(N, 128, x.H/8, x.W/8)
    self.layer2_1x1 = convrelu(128, 128, 1, 0)
    self.layer3 = self.base_layers[6]  # size=(N, 256, x.H/16, x.W/16)
    self.layer3_1x1 = convrelu(256, 256, 1, 0)
    self.layer4 = self.base_layers[7]  # size=(N, 512, x.H/32, x.W/32)
    self.layer4_1x1 = convrelu(512, 512, 1, 0)

    self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

    self.conv_up3 = convrelu(256 + 512, 512, 3, 1)
    self.conv_up2 = convrelu(128 + 512, 256, 3, 1)
    self.conv_up1 = convrelu(64 + 256, 256, 3, 1)
    self.conv_up0 = convrelu(64 + 256, 128, 3, 1)

    self.conv_original_size0 = convrelu(3, 64, 3, 1)
    self.conv_original_size1 = convrelu(64, 64, 3, 1)
    self.conv_original_size2 = convrelu(64 + 128, 64, 3, 1)

    self.conv_last = nn.Conv2d(64, n_class, 1)

  def forward(self, input):
    x_original = self.conv_original_size0(input)
    x_original = self.conv_original_size1(x_original)

    layer0 = self.layer0(input)
    layer1 = self.layer1(layer0)
    layer2 = self.layer2(layer1)
    layer3 = self.layer3(layer2)
    layer4 = self.layer4(layer3)

    layer4 = self.layer4_1x1(layer4)
    x = self.upsample(layer4)
    layer3 = self.layer3_1x1(layer3)
    x = torch.cat([x, layer3], dim=1)
    x = self.conv_up3(x)

    x = self.upsample(x)
    layer2 = self.layer2_1x1(layer2)
    x = torch.cat([x, layer2], dim=1)
    x = self.conv_up2(x)

    x = self.upsample(x)
    layer1 = self.layer1_1x1(layer1)
    x = torch.cat([x, layer1], dim=1)
    x = self.conv_up1(x)

    x = self.upsample(x)
    layer0 = self.layer0_1x1(layer0)
    x = torch.cat([x, layer0], dim=1)
    x = self.conv_up0(x)

    x = self.upsample(x)
    x = torch.cat([x, x_original], dim=1)
    x = self.conv_original_size2(x)

    out = self.conv_last(x)

    return out

In [ ]:
# #discriminator model
from torchsummary import summary

# import torch
# import torch.nn as nn

class Block(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 4, stride, 1, bias=True, padding_mode="reflect"),
            nn.InstanceNorm2d(out_channels),
            nn.LeakyReLU(0.2, inplace=True),
        )

    def forward(self, x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self, in_channels=3, features=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(
                in_channels,
                features[0],
                kernel_size=4,
                stride=2,
                padding=1,
                padding_mode="reflect",
            ),
            nn.LeakyReLU(0.2, inplace=True),
        )

        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(Block(in_channels, feature, stride=1 if feature==features[-1] else 2))
            in_channels = feature

        layers.append(nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"))
        self.model = nn.Sequential(*layers)


    def forward(self, x):
      x = self.initial(x)
      return torch.sigmoid(self.model(x))

# class Discriminator(nn.Module):
#     def __init__(self, in_channels=3, features=[64, 128, 256]):
#         super().__init__()
#         self.initial = nn.Sequential(
#             nn.Conv2d(
#                 in_channels,
#                 features[0],
#                 kernel_size=4,
#                 stride=2,
#                 padding=1,
#                 padding_mode="reflect",
#             ),
#             nn.LeakyReLU(0.2, inplace=True),
#         )

#         layers = []
#         in_channels = features[0]
#         for feature in features[1:]:
#             layers.append(Block(in_channels, feature, stride= 2))
#             in_channels = feature
#         layers.append(Block(in_channels, 512, kernel_size = ?, stride=1), pad=1?, bias = True,  padding_mode="reflect")
#         layers.append(nn.Conv2d(in_channels, 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect"))
#         self.model = nn.Sequential(*layers)

#     def forward(self, x):
#         x = self.initial(x)
#         return torch.sigmoid(self.model(x))

def test():
    x = torch.randn((5, 3, 256, 256))
    model = Discriminator(in_channels=3)
    # summary(model, (3,256,256))
    preds = model(x)


    print(preds.shape)


if __name__ == "__main__":
    test()


#from :  https://bjornkhansen95.medium.com/cycle-gan-with-pytorch-ebe5db947a99

# nc=3
# ndf=64
# class Discriminator(nn.Module):
#     def __init__(self):
#         super(Discriminator, self).__init__()
#         self.main = nn.Sequential(
#             # input is (nc) x 128 x 128
#             nn.Conv2d(nc,ndf,4,2,1, bias=False),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf) x 64 x 64
#             nn.Conv2d(ndf,ndf*2,4,2,1, bias=False),
#             nn.InstanceNorm2d(ndf * 2),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*2) x 32 x 32
#             nn.Conv2d(ndf*2, ndf * 4, 4, 2, 1, bias=False),
#             nn.InstanceNorm2d(ndf * 4),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*4) x 16 x 16
#             nn.Conv2d(ndf*4,ndf*8,4,1,1),
#             nn.InstanceNorm2d(ndf*8),
#             nn.LeakyReLU(0.2, inplace=True),
#             # state size. (ndf*8) x 15 x 15
#             nn.Conv2d(ndf*8,1,4,1,1),
#             # state size. 1 x 14 x 14
#         )
#         self.last_lin = nn.Linear(30*30,1)

#     def forward(self, input):
#         x = self.main(input)
#         x = x.view(-1, 30*30)
#         x = self.last_lin(x)
#         return x


# def test():
#     x = torch.randn((5, 3, 256, 256))
#     model = Discriminator()
#     preds = model(x)
#     print(preds.shape)


# if __name__ == "__main__":
#     test()

torch.Size([5, 1, 30, 30])


In [ ]:

class preprocessDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.img = os.listdir(root)
        self.length_dataset = self.len = len(self.img)

    def __len__(self):
        return self.len

    def __getitem__(self, index):
        img = self.img[index % self.len]
        path = os.path.join(self.root, img)
        img = np.array(Image.open(path))#.convert("RGB"))

        if self.transform:
            augmentations = self.transform(image=img)
            img = augmentations["image"]

        img =  np.moveaxis(img, -1,0).astype(float)  # change dimension shape : from  [256,256,3] to [3,256,256] to

        return img

In [ ]:
# preprocess generatoe

pre_BATCH_SIZE = 5
pre_LEARNING_RATE = 1e-5
pre_NUM_EPOCHS = 1
pre_LOAD_MODEL = False
pre_SAVE_MODEL = True
CHECKPOINT_GEN_U = path+"/genu.pth.tar"

def train_pre(gen, loader, opt_gen, l1, mse, g_scaler,j):

    for (idx,  picture) in enumerate(loader):
        # print(idx)

        picture = picture.to(DEVICE)

        # Train U-net
        with torch.cuda.amp.autocast():
            out_picture = gen(picture)
            picture_loss = mse(picture, out_picture)

        opt_gen.zero_grad()
        g_scaler.scale(picture_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()


        if idx % 100 == 0:
          if pre_SAVE_MODEL:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN_U)
          save_image(picture, path+f"/pretrain_g/original{idx}.png", normalize=True)
          save_image(out_picture, path+f"/pretrain_g/output{idx}.png", normalize=True)


def main():
    model = ResNetUNet(3)

    gen =  model.double().to(DEVICE)

    opt_gen = optim.Adam(list(gen.parameters()), lr=pre_LEARNING_RATE)
    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if pre_LOAD_MODEL:
        load_checkpoint(CHECKPOINT_GEN_U, gen, opt_gen, pre_LEARNING_RATE)
    q = preprocessDataset(path+"/photo_jpg", transform=transforms)
    l = len(q)
    # tr_dataset , val_dataset = torch.utils.data.random_split(q, [.8*l, .2*l], generator=torch.Generator().manual_seed(42))
    # val_loader = DataLoader(val_dataset,batch_size=1)
    tr_loader = DataLoader(q, batch_size=pre_BATCH_SIZE,shuffle=True)
    g_scaler = torch.cuda.amp.GradScaler()

    j=0
    for epoch in range(pre_NUM_EPOCHS):
        j+=1
        start = time.time()
        print('epoch', epoch+1 , 'over',pre_NUM_EPOCHS )
        train_pre(gen, tr_loader, opt_gen, L1, mse, g_scaler,j)
        end = time.time()
        print("time for this epoch:", int((end-start)/60),"min",(end-start)%60,"secs")

        if pre_SAVE_MODEL:
            save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN_U)


if __name__ == "__main__":
    # torch.cuda.empty_cache()
    # seed_everything()

    # main()

    print('end')

end


In [ ]:
# #train
# import torch
# import time
# # from dataset import PictureMonetDataset
# import sys
# # from utils import save_checkpoint, load_checkpoint
# from torch.utils.data import DataLoader
# import torch.nn as nn
# import torch.optim as optim
# # import config
# from tqdm import tqdm
# from torchvision.utils import save_image
# # from discrimator_model import Discriminator
# # from generator_model import Generator

def train_fn(disc_P, disc_M, gen_M, gen_P, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler,j):
    dmr = []
    dmrl = []
    dmf = []
    dmfl = []
    dpr = []
    dpf = []
    loss_g = []
    loss_d = []
    P_reals = 0
    P_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (monet, picture) in enumerate(loop):
        monet = monet.to(DEVICE)
        picture = picture.to(DEVICE)
        # save_image(monet, "/content/first_in_train.png", normalize= True)

        # yo= monet.detach().numpy()
        # img = Image.fromarray(yo[0], 'RGB')
        # img.save('stam.png')
        # img.show()


        # Train Discriminators P and M
        # with torch.cuda.amp.autocast():

        fake_monet = gen_M(picture)
        D_M_real = disc_M(monet)

        D_M_fake = disc_M(fake_monet.detach())
        D_M_real_loss = mse(D_M_real, torch.ones_like(D_M_real))

        D_M_fake_loss = mse(D_M_fake, torch.zeros_like(D_M_fake))
        D_M_loss = D_M_real_loss + D_M_fake_loss


        dmf.append(float(torch.mean(D_M_fake)))
        dmfl.append( float(D_M_fake_loss))
        dmr.append(float(torch.mean(D_M_real)))
        dmrl.append(float(D_M_real_loss))

        fake_picture = gen_P(monet)
        D_P_real = disc_P(picture)
        D_P_fake = disc_P(fake_picture.detach())
        P_reals += D_P_real.mean().item()
        P_fakes += D_P_fake.mean().item()
        D_P_real_loss = mse(D_P_real, torch.ones_like(D_P_real))
        D_P_fake_loss = mse(D_P_fake, torch.zeros_like(D_P_fake))
        D_P_loss = D_P_real_loss + D_P_fake_loss

        # print(torch.mean(D_P_fake))
        # print(dpf)

        dpf.append(float(torch.mean(D_P_fake)))
        dpr.append(float(torch.mean(D_P_real)))
        # put it togethor
        D_loss = (D_P_loss + D_M_loss)/2

        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()

        # Train Generators P and M
        # with torch.cuda.amp.autocast():
        # adversarial loss for both generators
        D_P_fake = disc_P(fake_picture)
        D_M_fake = disc_M(fake_monet)
        # print("D_P_fake", torch.mean(D_P_fake))
        # print("D_M_fake", torch.mean(D_M_fake))
        loss_G_P = mse(D_P_fake, torch.ones_like(D_P_fake))
        loss_G_M = mse(D_M_fake, torch.ones_like(D_M_fake))

        # cycle loss
        cycle_monet = gen_M(fake_picture)
        cycle_picture = gen_P(fake_monet)
        cycle_monet_loss = l1(monet, cycle_monet)
        cycle_picture_loss = l1(picture, cycle_picture)

        # identity loss (remove these for efficiency if you set lambda_identity=0)
        identity_monet = gen_M(monet)
        identity_picture = gen_P(picture)
        identity_monet_loss = l1(monet, identity_monet)
        identity_picture_loss = l1(picture, identity_picture)

        # add all togethor
        G_loss = (
                loss_G_M
                + loss_G_P
                + cycle_monet_loss * LAMBDA_CYCLE
                + cycle_picture_loss *LAMBDA_CYCLE
                + identity_picture_loss * LAMBDA_IDENTITY *0.5
                + identity_monet_loss * LAMBDA_IDENTITY * 0.5
        )
        loss_d.append(D_loss)
        loss_g.append(G_loss)

        # opt_gen.zero_grad()
        # g_scaler.scale(G_loss).backward()
        # g_scaler.step(opt_gen)
        # g_scaler.update()
        j+=10


        if idx % 300 == 0:


            save_image(picture, path+f"/saved_images/picture_{idx}.png", normalize=True)
            # save_image(picture * 0.5 + 0.5, path+f"/saved_images/picture_{j}.png", normalize=True)
            save_image(monet , path+f"/saved_images/monet_{idx}.png", normalize=True)
            save_image(fake_picture , path+f"/saved_images/fp_Gp(m)_{idx}.png", normalize=True)
            save_image(fake_monet, path+f"/saved_images/fm_Gm(p)_{idx}.png", normalize=True)
            # save_image(fake_monet* 0.5 + 0.5 , path+f"/saved_images/fm_Gm(p)_+demi{idx}.png", normalize=True)

            # save_image(cycle_monet , path+f"/saved_images/cycle_monet{idx}.png", normalize=True)
            # save_image(cycle_picture , path+f"/saved_images/cycle_picture{idx}.png", normalize=True)

            save_image(identity_monet , path+f"/saved_images/identity_monet{idx}.png", normalize=True)
            save_image(identity_picture , path+f"/saved_images/identity_picture{idx}.png", normalize=True)


        loop.set_postfix(H_real=P_reals/(j+1), H_fake=P_fakes/(j+1))

    print('dmf',dmf)
    print('dmfl', dmfl)
    print('dmr',dmr)
    print('dmrl', dmrl)
    print('for this epoch , dmf',Average(dmf),', dmr',Average(dmr),', dpf',Average(dpf), 'dpr',Average(dpr),"loss gen",Average(loss_g), 'loss disc', Average(loss_d) )

    # print(dpf)
    # print('for this epoch , dmf',Average(dmf),', dmr',Average(dmr),', dpf',Average(dpf), 'dpr',Average(dpr))


def main():
    d0  = Discriminator()
    weights_init_normal(d0)
    d = d0.double().to(DEVICE) # old v : in discr param : in_channels=3
    disc_P = d
    disc_M = d
    model = ResNetUNet(3)
    weights_init_normal(model)
    gen_M = model.double().to(DEVICE)
    gen_P = model.double().to(DEVICE)

    # model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',  in_channels=3, out_channels=3, init_features=32, pretrained=True)
    # gen_M = model().double().to(DEVICE)
    # gen_P = model().double().to(DEVICE)


    # gen_M = Generator(img_channels=3, num_residuals=9).double().to(DEVICE)# old v: img_channels=3, num_residuals=9
    # gen_P = Generator(img_channels=3, num_residuals=9).double().to(DEVICE)

    opt_disc = optim.Adam(
            list(disc_P.parameters()) + list(disc_M.parameters()),
            lr=LEARNING_RATE_D)#, weight_decay=WD)


    # opt_disc_temp = optim.Adam(
    #     list(disc_P.parameters()) ,
    #     lr=LEARNING_RATE)#, weight_decay=WD)

    # opt_gen_temp = optim.Adam(
    #     list(gen_M.parameters()),
    #     lr=LEARNING_RATE)#, weight_decay=WD)

    opt_gen = optim.Adam(
        list(list(gen_P.parameters())),
        lr=LEARNING_RATE_G)#, weight_decay=WD)

    L1 = nn.L1Loss()
    mse = nn.MSELoss()
    if PRE_G:
      load_checkpoint(CHECKPOINT_GEN_U, gen_M, opt_gen, pre_LEARNING_RATE)

      load_checkpoint(CHECKPOINT_GEN_U, gen_P, opt_gen, pre_LEARNING_RATE,
      )



    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_H, gen_P, opt_gen, LEARNING_RATE_G,
        )
        load_checkpoint(
            CHECKPOINT_GEN_Z, gen_M, opt_gen, LEARNING_RATE_G,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_H, disc_P, opt_disc, LEARNING_RATE_D,
        )
        load_checkpoint(
            CHECKPOINT_CRITIC_Z, disc_M, opt_disc, LEARNING_RATE_D
        )

    dataset = PictureMonetDataset(
        root_picture= path +"/300_pic", root_monet= path +"/aug_monet", transform=transforms    )
    # val_dataset = PictureMonetDataset(
    #     root_picture=VAL_DIR+"/pictures",  root_monet=VAL_DIR+"/monets" ,transform=transforms)
    # val_loader = DataLoader(
    #     val_dataset,
    #     batch_size=1,
    #     shuffle=False,
    #     pin_memory=True
    # )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True

    )
    g_scaler = torch.cuda.amp.GradScaler()
    d_scaler = torch.cuda.amp.GradScaler()
    j=0
    for epoch in range(NUM_EPOCHS):
        j+=1
        start = time.time()
        print('epoch', epoch+1 , 'over',NUM_EPOCHS )
        train_fn(disc_P, disc_M, gen_M, gen_P, loader, opt_disc, opt_gen, L1, mse, d_scaler, g_scaler,j)
        end = time.time()
        print("time for this epoch:", int((end-start)/60),"min",(end-start)%60,"secs")

        if SAVE_MODEL:
            save_checkpoint(gen_P, opt_gen, filename=CHECKPOINT_GEN_H)
            save_checkpoint(gen_M, opt_gen, filename=CHECKPOINT_GEN_Z)
            save_checkpoint(disc_P, opt_disc, filename=CHECKPOINT_CRITIC_H)
            save_checkpoint(disc_M, opt_disc, filename=CHECKPOINT_CRITIC_Z)


     #checking
    # loop = tqdm(loader, leave=True)

    # for idx, (monet, picture) in enumerate(loop):
    #     # monet = monet.to(DEVICE)
    #     # picture = picture.to(DEVICE)
    #     monet =  torch.moveaxis(monet, 1, -1)
    #     n_monet = torch.squeeze(monet).type(torch.FloatTensor)
    #     print('here', n_monet.size())
    #     save_image(n_monet, f"/content/first_in_train{idx}.png")




if __name__ == "__main__":
    torch.cuda.empty_cache()
    seed_everything()

    main()

    print('end')

RuntimeError: ignored